In [5]:
import pandas as pd
import numpy as np
from pydantic import Field, BaseModel
from typing import Union

In [34]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.tools import Tool
from pprint import pprint
from langchain.agents import initialize_agent, AgentType


db = SQLDatabase.from_uri(
    "sqlite:///pdp_funczone.sqlite",
)
llm = ChatOpenAI(temperature=0)

# Create a prompt to load table names and and column names to database
prompt = PromptTemplate.from_template(
    "You are a helpful assistant that provides SQL queries based on user requests. "
    "Search for the table names and columns in the database: {table_context}. "
    "When asked for a query, you will return a valid SQL query that answers the user's question."
)
prompt.invoke({"table_context": db.get_context()})



def zone_name_lookup(query):
    import pandas as pd
    df = pd.read_excel("order_505_sub.xlsx")
    # Simplified filter logic
    zone_name = df[df["code"] == int(query)]["name"].values[0]
    return zone_name

lookup_tool = Tool(
    name="Zone name lookup",
    func=zone_name_lookup,
    description="Look up zone name by zone code after you god zone code from database",
)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

sql_agent_executor = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True)



sql_agent_tool = Tool(
    name="SQL Agent",
    func=sql_agent_executor.run,
    description="Use this tool to run SQL queries against the table pdp_functional_zones__default_workspacepdp_functional_zones of a database to find pdp_func_zone_code WHERE ID is  supplied by user ",
)

tools = [lookup_tool, sql_agent_tool]

multi_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

multi_agent.run("What is the name of zone by id of 2726883")



> Entering new AgentExecutor chain...
We need to find the zone name corresponding to the zone ID 2726883.
Action: SQL Agent
Action Input: SELECT pdp_func_zone_code FROM pdp_functional_zones__default_workspacepdp_functional_zones WHERE ID = 2726883

> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: geometry_columns, pdp_functional_zones__default_workspacepdp_functional_zones, spatial_ref_sysThe table I need to query is pdp_functional_zones__default_workspacepdp_functional_zones. I should check the schema of this table.
Action: sql_db_schema
Action Input: pdp_functional_zones__default_workspacepdp_functional_zones
CREATE TABLE pdp_functional_zones__default_workspacepdp_functional_zones (
	ogc_fid INTEGER, 
	"WKT_GEOMETRY" VARCHAR, 
	id INTEGER, 
	area FLOAT, 
	creation_doc_id INTEGER, 
	gp_func_zone_code_id INTEGER, 
	pdp_func_zone_code_id INTEGER, 
	deactivation_doc_id INTEGER, 
	zone_type_id INTEGER, 
	func_zone_status_id INTEGER, 
	pdp_func_zone_cod

'The name of the zone with ID 2726883 is **Территория прочих объектов социального и культурно-бытового обслуживания населения.'